In [1]:
import os
import pandas as pd
import numpy as np
import PyPDF2
from openpyxl import Workbook

In [2]:
BUDGET_CONCEPT_FORMAT = 'code'

In [3]:
pdf_file = open('21.616 PE-Mediciones.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
pages_num = len(pdf_reader.pages)


In [4]:
# pdf_file = open('prueba_2.pdf', 'rb')
# pdf_reader = PyPDF2.PdfFileReader(pdf_file)
# pages_num = pdf_reader.numPages

In [5]:
pdf_reader.page_layout

In [6]:
REPLACEMENTS = [('Demolició n', 'Demolición'),
                ('elem ento', 'elemento'),
                ('prueba s', 'pruebas'),
                ('saneami ento','saneamiento'),
                ('revo co','revoco'),
                ('gar aje','garaje'),
                (' - ',''),
                (' -',''),
                ('- ',''),
                ('horizont al','horizontal'),
                ('ladrillo s','ladrillos'),
                ('mecá nica','mecánica'),
                ('espon jamientos','esponjamientos'),
                ('de jará','dejará'),
                ('d e','de'),
                ('fi lm','film'),
                ('formació n','formación'),
                ('em plear','emplear'),
                ('galv anizado','galvanizado'),
                ('betú n','betún'),
                ('aplicació n','aplicación'),
                ('c olocación','colocación'),
                ('colo cación','colocación'),
                ('co locación','colocación'),
                ('especif icaciones','especificaciones'),
                ('medició n','medición'),
                ('co n','con'),
                ('alineacione s','alineaciones'),
                ('m arco','marco'),
                ('fijació n','fijación'),
                ('exte rior','exterior'),
                ('tapajun tas','tapajuntas'),
                ('humed ad','humedad'),
                ('buzó n','buzón'),
                ('co mplementarios','complementarios'),
                ('parame nto','paramento'),
                ('singulare s','singulares'),
                ('inclu ir','incluir'),
                ('tuberí a','tubería')]

In [7]:
def convert_number(i, string):
    # Converts coma separated numbers in string format to float
    string = string.replace('.','').replace(',','.')
    try:
        result = float(string)
    except:
        raise Exception(f"String {string} can't be converted to float in line {i}")
    return result

def is_number(string):
    """
    Returns true if string is a digit with point separated or coma separated decimals
    """
    string = string.strip()
    if string[0] == '-':
        string = string[1:]
    string = string.replace('.','')
    string = string.replace(',','')
    return string.isdigit()

def add_description(original, new):
    # Combines both lines into one string with no extra spaces
    return f"{original.strip()} {new.strip()}"

def check_spelling(description, replacements):
    """
    Corrects the words that appear in the replacementes list of the description strig.
    Args:
     description: String with the text to correct.
     replacements: List of tuples with the original, and replacement words.
    """
    for replacement in replacements:
        index = description.lower().find(replacement[0].lower())
        if index == -1:
            continue
        first_index = None
        last_index = None
        if len(replacement[1]) == 0:
            description = description.replace(replacement[0], replacement[1])
        else:
            for i in range(len(replacement[0])):
                if replacement[0][i].lower() == replacement[1][i].lower():
                    continue
                first_index = i
                break
            for i in range(-1, -len(replacement[0]), -1):
                if replacement[0][i].lower() == replacement[1][i]:
                    continue
                last_index = i + 1
                break
            description = description[:index+first_index] + description[-( len(description) - index - len(replacement[0]) - last_index ):]
    return description


In [8]:
pageObj = pdf_reader.pages[55]
text = pageObj.extract_text()
lines = text.split('\n')
lines[48].strip().split()[-1][-1]



'2'

In [9]:
presupuesto = []

headers_lines = None
partial_measurements_header = ''

capitulo_abierto = False
description_open = False
partial_measurements_open = False
final_measurement_open = False
line_type = None
c=0

for i in range(pages_num):
    pageObj = pdf_reader.pages[i]
    text = pageObj.extract_text()
    print()
    print(f"PAGE: {i+1}")
    for ii, line in enumerate(text.split('\n')):
        c += 1

        # if line.strip() == "":
        #     line_type = 'null_line'
        # elif headers_lines is not None and ii < headers_lines:
        #     line_type = 'headers_line'
        # elif line.lstrip().startswith('CAPÍ') or line.lstrip().startswith('CAPI'):


        if line.strip() == "":
            # print(f"{c-1}: {line}")
            continue

        if headers_lines is not None and ii < headers_lines:
            # print(f"ND - {c-1}: {line}")
            continue

        
        if line.lstrip().startswith('CAPÍ') or line.lstrip().startswith('CAPI'):
            # New budget chapter starts
            if capitulo_abierto:
                presupuesto.append(capitulo)
            else:
                headers_lines = ii
                print(f"HEADERS LINES ARE: {headers_lines} ------------------------")
            capitulo_abierto = True
            capitulo = dict() # Num, Name, Concepts
            line_split = line.split()
            capitulo['NUM'] = [block for block in line_split if block.isdigit()][0]
            capitulo['NAME'] = ' '.join(line_split[line_split.index(capitulo['NUM'])+1:])
            capitulo['CONCEPTS'] = []

            print(f"@CAPÍTULO:    {c-1}: {line}")
            continue

        if headers_lines is None:
            print(f"@HEADERS:     {c-1}: {line}")
            continue

        if  (BUDGET_CONCEPT_FORMAT == 'mayusc'
            and line.strip().split()[-1] == line.strip().split()[-1].upper() 
            and not line.strip().split()[-1][-1].isdigit()
            and not description_open):
            # New budget concept starts with mayusc type
            
            partida = dict() # Número Partida, Unidad, Título, Descripción, Mediciones parciales (lista), Medición
            mediciones_parciales = [] # New list of partial measures
            medicion_parcial = dict() # Nombre, Medición Parcial
            
            line_split = line.split()
            description_open = True

            partida['ID'] = c
            unit_list = [block for j, block in enumerate(line_split) if j == 0 or (j == 1 and len(block) == 1)]
            unit = ''.join(unit_list)
            partida['Unit'] = unit
            partida['TITLE'] = ' '.join(line_split[len(unit_list):]).replace('  ', ' ').replace('  ',' ')
            partida['DESCRIPTION'] = ''

            print('________________________________________________________')
            print(f"@PARTIDA:     {c-1}: {line}")
            continue

        if  (BUDGET_CONCEPT_FORMAT == 'code' 
            and line.lstrip()[:2].isdigit() 
            and line.lstrip()[2] == '.' 
            and line.lstrip()[3:5].isdigit()
            and not description_open):
            # New budget concept starts with code type
            
            partida = dict() # Número Partida, Unidad, Título, Descripción, Mediciones parciales (lista), Medición
            mediciones_parciales = [] # New list of partial measures
            medicion_parcial = dict() # Nombre, Medición Parcial
            
            line_split = line.split()
            description_open = True

            partida['ID'] = line_split[0]
            partida['Unit'] = line_split[1]
            partida['TITLE'] = ' '.join(line_split[2:]).replace('  ', ' ').replace('  ',' ')
            partida['DESCRIPTION'] = ''

            print('________________________________________________________')
            print(f"@PARTIDA:     {c-1}: {line}")
            continue

        if description_open and not line.rstrip().endswith('.') and len([block for block in line.split() if not is_number(block)]) < 6:
            # End concept description
            description_open = False
            partial_measurements_open = True
            if not is_number(line.strip().split()[-1]):
                # Line is a chapter of partial measurements
                partial_measurements_header = line.strip()
                print(f"@PART.MEAS.:  {c-1}: {line}")
                continue

            partial_measurements_name = ' '.join([block for j, block in enumerate(line.strip().split()) if j == 0 or not block[0].isdigit()])
            medicion_parcial['NAME'] = f"{partial_measurements_header} - {partial_measurements_name}" if partial_measurements_header != '' else partial_measurements_name
            medicion_parcial['MEASUREMENT'] = convert_number(c, line.strip().split()[-1])
            mediciones_parciales.append(medicion_parcial.copy())

            print(f"@PART.MEAS.:  {c-1}: {line}")
            continue

        if description_open:
            # Add new line to concept description
            partida['DESCRIPTION'] = add_description(partida['DESCRIPTION'], line)

            print(f"@DESCRIPCIÓN: {c-1}: {line}")
            continue

        if partial_measurements_open and line.strip()[0] == '_' and line.strip()[-1] == '_':
            # Partial measurements ends and budget concept ends
            partial_measurements_open = False
            final_measurement_open = True
            continue

        if final_measurement_open:
            final_measurement_open = False
            partial_measurements_header = ''
            partida['PARTIAL MEASUREMENTS'] = mediciones_parciales
            partida['MEASURE'] = convert_number(c,line)
            partida['DESCRIPTION'] = check_spelling(partida['DESCRIPTION'], REPLACEMENTS)
            partida['TITLE'] = check_spelling(partida['TITLE'], REPLACEMENTS)
            capitulo['CONCEPTS'].append(partida)

            print(f"@TOTAL MEAS.: {c}: {line}")
            continue

        if partial_measurements_open:
            # Add new partial measurement
            if not is_number(line.strip().split()[-1]):
                # Line is a chapter of partial measurements
                partial_measurements_header = line.strip()
                # print(f"@PART.MEAS.:  {c-1}: {line}")
                continue
            
            # Get name of partial measurement
            partial_measurements_name = ' '.join([block for j, block in enumerate(line.strip().split()) if j == 0 or not block[0].isdigit()])
            medicion_parcial['NAME'] = f"{partial_measurements_header} - {partial_measurements_name}" if partial_measurements_header != '' else partial_measurements_name

            medicion_parcial['MEASUREMENT'] = convert_number(c, line.strip().split()[-1])
            
            mediciones_parciales.append(medicion_parcial.copy())
            # print(mediciones_parciales)

            print(f"@PART.MEAS.:  {c-1}: {line}")
            continue
        
        print(f"ND - {c-1}: {line}")
            

        
    # if i == 4:
    #     break

presupuesto.append(capitulo)
    


PAGE: 1
@HEADERS:     0: REFORMA Y AMPLIACI ÓN DE DOS VIVIENDAS PAREAD AS Camino de Aucanada nº79, Urb Aucanada - TM ALCUDIA   MEMORIA  
@HEADERS:     3: CÓDIGO  RESUMEN  UDS  LONGITUD  ANCHURA  ALTURA  PARCIALES  CANTIDAD  
@HEADERS:     4:  ________________________________ ________________________________ ________________________________ ________________________________ ___________   
@HEADERS:     6: MIGUEL SUÑ ER COMAS - MIGUEL L. CERDÁ TORRES – RODRIGO MARTINEZ FLECHOSO   ARQUITECTOS  
HEADERS LINES ARE: 7 ------------------------
@CAPÍTULO:    7:   CAPÍ TULO 01 Demoliciones                                                      
________________________________________________________
@PARTIDA:     8: 01.01  m²   Demolición de muro de marés                                       
@DESCRIPCIÓN: 9:  Demolició n de hoja exterior en cerramiento de fachada, de fábrica de marés, de 20 cm de espesor,   
@DESCRIPCIÓN: 10:  con medios manuales, sin afectar a la estabilidad de los elementos 

In [10]:
presupuesto[5]['CONCEPTS'][1]['DESCRIPTION']
# REPLACEMENTS = [(' - ','')]
# for replacement in REPLACEMENTS:
#         index = description.lower().find(replacement[0].lower())
#         if index == -1:
#             continue
#         first_index = None
#         last_index = None
#         if len(replacement[1]) == 0:
#             first_index = 0
#             last_index = 0
#         else:
#             for i in range(len(replacement[0])):
#                 if replacement[0][i].lower() == replacement[1][i].lower():
#                     continue
#                 first_index = i
#                 break
#             for i in range(-1, -len(replacement[0]), -1):
#                 if replacement[0][i].lower() == replacement[1][i]:
#                     continue
#                 last_index = i + 1
#                 break
#         description = description[:index+first_index] + description[-( len(description) - index - len(replacement[0]) - last_index ):]
# print(description)

'Formación de cubierta plana transitable, no ventilada, tipo convencional, pendiente del 1% al 5%, para tráfico peatonal privado, compuesta de los siguientes elementos: BARRERA DE VAPOR: lámina de oxiasfalto, LO30-PE, masa nominal 3 kg/m², con armadura de film de polietileno de 95 g/m², acabada con film plástico termofusible en ambas caras colocada con emulsión asfáltica aniónica sin cargas, tipo EA; AISLAMIENTO TÉRMICO: panel rígido depoliestireno extruido machihembrado tipo STYROFOAM ROOFMATE o si milar, de 80 mm de espesor; FORMACIÓN DE PENDIENTES: mediante encintado de limatesas, limahoyas y juntas con maestras de ladrillo cerámico hueco doble y capa de 10 cm de espesor medio a base de arcilla expandida de 350 kg/m³ de densidad, vertida en seco y consolidada en su superficie con lechada de cemento, proporcionando una resistencia a compresión de 1 MPa y con una conductividad térmica de 0,087 W/(mK); acabado con capa de regularización de mortero de cemento, industrial, M5 de 4 cm de 

In [11]:
# Comprobación de presupuesto
for cap in presupuesto:
    print('--------',cap['NUM'], cap['NAME'])
    for concept in cap['CONCEPTS']:
        print('--------',concept['ID'], concept['TITLE'], concept['Unit'], concept['MEASURE'])
        print(f"{concept['DESCRIPTION']}")
        for meas in concept['PARTIAL MEASUREMENTS']:
            print(meas['NAME'], meas['MEASUREMENT'])

-------- 01 Demoliciones
-------- 01.01 Demolición de muro de marés m² 354.65
Demolición de hoja exterior en cerramiento de fachada, de fábrica de marés, de 20 cm de espesor, con medios manuales, sin afectar a la estabilidad de los elementos constructivos contiguos, y carga manual sobre camión o contenedor. Incluye: Demolición del elemento. Fragmentación de los escombros en piezas manejables. Retirada y acopio de escombros. Limpieza de los restos de obra. Carga manual de escombros sobre camión o contenedor.
Vivienda A - PB 10.15
Vivienda A - 1 10.01
Vivienda A - 1 14.595
Vivienda A - 1 2.695
Vivienda A - 1 39.445
Vivienda A - 1 24.85
Vivienda A - 1 28.0
Vivienda A - 1 3.5
Vivienda A - 1 9.905
Vivienda A - 1 14.0
Vivienda A - 1 11.55
Vivienda A - 1 23.1
Vivienda A - 1 35.75
Vivienda B - PB 24.5
Vivienda B - 1 14.35
Vivienda B - 1 38.5
Vivienda B - 1 22.75
Vivienda B - 1 27.0
-------- 01.02 Demolición de partición interior de fábrica de marés m² 159.635
Demolición de partición interior d

In [12]:
wb = Workbook()
ws = wb.active
c = 1
ws['B1'] = 'CAP NUM'
ws['C1'] = 'CAPÍTULO'
ws['D1'] = 'ID'
ws['E1'] = 'PARTIDA'
ws['F1'] = 'UNIDAD'
ws['G1'] = 'MEDIDA'
ws['H1'] = 'DESCRIPCIÓN'
ws['I1'] = 'ZONA'
ws['J1'] = 'MEDIDA PARCIAL'
for capitulo in presupuesto:
    print('----', capitulo['NUM'], capitulo['NAME'])
    for concept in capitulo['CONCEPTS']:
        print('----', concept['ID'], concept['TITLE'], concept['Unit'], concept['MEASURE'])
        print(concept['DESCRIPTION'])
        for measurement in concept['PARTIAL MEASUREMENTS']:
            print(measurement['NAME'], measurement['MEASUREMENT'])
            c += 1
            ws[f'B{c}'] = capitulo['NUM']
            ws[f'C{c}'] = capitulo['NAME']
            ws[f'D{c}'] = concept['ID']
            ws[f'E{c}'] = concept['TITLE']
            ws[f'F{c}'] = concept['Unit']
            ws[f'G{c}'] = concept['MEASURE']
            ws[f'H{c}'] = concept['DESCRIPTION']
            ws[f'I{c}'] = measurement['NAME']
            ws[f'J{c}'] = measurement['MEASUREMENT']

wb.save('sample.xlsx')


---- 01 Demoliciones
---- 01.01 Demolición de muro de marés m² 354.65
Demolición de hoja exterior en cerramiento de fachada, de fábrica de marés, de 20 cm de espesor, con medios manuales, sin afectar a la estabilidad de los elementos constructivos contiguos, y carga manual sobre camión o contenedor. Incluye: Demolición del elemento. Fragmentación de los escombros en piezas manejables. Retirada y acopio de escombros. Limpieza de los restos de obra. Carga manual de escombros sobre camión o contenedor.
Vivienda A - PB 10.15
Vivienda A - 1 10.01
Vivienda A - 1 14.595
Vivienda A - 1 2.695
Vivienda A - 1 39.445
Vivienda A - 1 24.85
Vivienda A - 1 28.0
Vivienda A - 1 3.5
Vivienda A - 1 9.905
Vivienda A - 1 14.0
Vivienda A - 1 11.55
Vivienda A - 1 23.1
Vivienda A - 1 35.75
Vivienda B - PB 24.5
Vivienda B - 1 14.35
Vivienda B - 1 38.5
Vivienda B - 1 22.75
Vivienda B - 1 27.0
---- 01.02 Demolición de partición interior de fábrica de marés m² 159.635
Demolición de partición interior de fábrica de